# Transformers


### Objective: Build different transformer components and test them.

In [49]:
import numpy as np
import string
import time
import torch
import pdb
import math
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
np.random.seed(124)



### I am using the convention of having the batch dimension first so tensors will have shapes of `(N, L, D)` where `N` is the batch dimension, `L` is the max sequence length, and `D` is the feature dimension.
The default in PyTorch is for the sequence dimension to be first, i.e., `(L, N, D)` but most functions in PyTorch can be altered to make the batch dimension to be first by using `batch_first=True`.


# Positional Encoder

The positional encoder is a simple function that takes a 3D tensor of shape (batch_size, sequence_length, encoding_size), i.e., `(N, L, D)`, and returns a 3D tensor of the same shape where positional encoding embedding has been added. The positional encoder is a function of the position of the token in the sequence. The positional encoder is defined as:

$$PE_{(pos,2i)} = \sin(pos/10000^{2i/d_{model}})$$

$$PE_{(pos,2i+1)} = \cos(pos/10000^{2i/d_{model}})$$

where $pos$ is the position and $i$ is the dimension. That is, each dimension of the positional encoding corresponds to a sinusoid. The wavelengths form a geometric progression from $2\pi$ to $10000 \cdot 2\pi$.

In practice, the positional encoding is added to the embedding vector. This is done by first creating a tensor of shape (1, sequence_length, d_model) and then adding it to the embedding vector. This ensures that the positional encoding is added to every element in the batch via broadcasting.

In [50]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len = 80):
        super().__init__()
        self.d_model = d_model
        
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(max_seq_len, d_model)

        # Loop over the positions and the embedding dimensions
        # and calculate the positional encoding for each dimension
        # and position

        for i in range(max_seq_len):
          for j in range(d_model):
            if j % 2:
              pe[i][j] += np.cos(i / (10000 ** (2*j/d_model)))
            else:
              pe[i][j] += np.sin(i / (10000 ** (2*j/d_model)))
        
        pe = pe.unsqueeze(0)

        self.register_buffer('pe', pe)
 
    
    def forward(self, x):
        # make embeddings relatively larger than pe
        x = x * math.sqrt(self.d_model)
        # add constant positional encoding to embedding
        seq_len = x.size(1)
        x = x + Variable(self.pe[:,:seq_len], requires_grad=False)
        return x

pe = PositionalEncoder(512)
input_pe = torch.arange(1, 513)*0.01
input_pe = input_pe.repeat(1, 4, 1).float()

output_pe = pe(input_pe)
print(output_pe.shape)

# check the difference between the two embeddings
print(torch.equal(input_pe, output_pe)) # They should not be equal after adding positional encoding
print(f"input_pe: {input_pe[0, 0, 0:5]} \noutput_pe: {output_pe[0, 0, 0:5]}")
print(f"input_pe: {input_pe[0, 1, 0:5]} \noutput_pe: {output_pe[0, 1, 0:5]}")
print(f"input_pe: {input_pe[0, 2, -5:]} \noutput_pe: {output_pe[0, 2, -5:]}")

torch.Size([1, 4, 512])
False
input_pe: tensor([0.0100, 0.0200, 0.0300, 0.0400, 0.0500]) 
output_pe: tensor([0.2263, 1.4525, 0.6788, 1.9051, 1.1314])
input_pe: tensor([0.0100, 0.0200, 0.0300, 0.0400, 0.0500]) 
output_pe: tensor([1.0677, 1.0222, 1.4808, 1.5285, 1.8931])
input_pe: tensor([5.0800, 5.0900, 5.1000, 5.1100, 5.1200]) 
output_pe: tensor([115.9473, 115.1735, 116.3998, 115.6261, 116.8524])


# Scaled Dot-Product Attention
I will implement a version of attention used in transformers:
$$
A(Q,K,V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V
$$
where $Q \in \mathbb{R}^{L\times d_k}$, $K \in \mathbb{R}^{L \times d_k}$, and $V \in \mathbb{R}^{L \times d_v}$, where $d_v$ is the dimension of the values. The softmax is across the column dimension. The output of attention should be a matrix $A \in \mathbb{R}^{L \times d_v}$.

Additionally, I will implement a **batched version** of this that can be used for multiple sequences at the same time. To do this, I will need to use the following **batched** version(s) of matrix multiplication either [`torch.bmm`](https://pytorch.org/docs/stable/generated/torch.bmm.html) or [`torch.matmul`](https://pytorch.org/docs/stable/generated/torch.matmul.html) for both the $QK^T$ and the product of the attention matrix and $V$. We recommend that you use `torch.bmm` as it is more explicit.

In [51]:
def attention(q, k, v):
    '''
    Inputs:
    q: query vector of shape (batch_size, seq_len, d_k)
    k: key vector of shape (batch_size, seq_len, d_k)
    v: value vector of shape (batch_size, seq_len, d_v)

    Returns:
    output: attention weighted sum of the value vectors 
        of shape (batch_size, seq_len, d_k)
    scores: attention weights of shape (batch_size, seq_len, seq_len)
    '''        
    d_k = k.size(-1)
    assert d_k == q.size(-1), 'q and k should have the same dimensionality'
    d_v = v.size(-1)

    K_trans = k.transpose(1, 2)

    val = torch.bmm(q, K_trans) / np.sqrt(d_k)

    att_values = torch.softmax(val, 2)
    output = torch.bmm(att_values, v)

    return output, att_values

# test the attention function with some random values
torch.manual_seed(42)
q = torch.randn(2, 5, 512)
k = torch.randn(2, 5, 512)
v = torch.randn(2, 5, 256)

output, att_values = attention(q, k, v)
print(f"Is shape of output correct? {output.shape == v.shape}")
print(f"Is shape of att_values correct? {att_values.shape == torch.Size([q.shape[0], q.shape[1], q.shape[1]])}")
print(f"Do attention values sum to 1? {torch.allclose(torch.sum(att_values, dim=-1), torch.ones(1))}")

# Last 25 values of last sample and last token
print(f"Output check (first): \n{output[0,0,:25]}")
print(f"Output check (last): \n{output[-1,-1,-25:]}")


Is shape of output correct? True
Is shape of att_values correct? True
Do attention values sum to 1? True
Output check (first): 
tensor([-1.3709, -0.6827,  0.3234,  0.8677, -0.1474, -0.9653, -0.7344,  0.8126,
         0.1219,  0.3224,  0.6257, -0.0958, -0.1664, -0.0667, -0.2810,  0.3068,
        -0.7030, -0.6719,  0.4364, -1.0071,  0.3534,  0.3160,  0.0326, -0.7315,
        -0.5165])
Output check (last): 
tensor([-0.2094,  1.3784,  0.2855, -0.1716,  0.1597, -0.6656,  0.3981, -0.9903,
        -0.6043, -0.6398,  0.0563, -1.5367, -0.0225, -0.8317,  0.0572,  0.2014,
         0.1324, -0.4563,  0.3832,  0.1051,  0.0653, -0.2076,  0.6225, -0.4946,
        -0.2935])


# Attention modules

## Self-attention module
I implement a self-attention module that takes in `x` and computes `q`,`k`,`v` internally using 3 linear layers. Then, I use the function from above to compute the output and attention and return it. The attention module takes as constructor parameters the `input_dim`, `key_dim`, and the `output_dim`.

In [52]:
class SelfAttention(nn.Module):
    def __init__(self, input_dim, key_dim, output_dim):
        super().__init__()

        self.lin_q = nn.Linear(input_dim, key_dim)
        self.lin_k = nn.Linear(input_dim, key_dim)
        self.lin_v = nn.Linear(input_dim, output_dim)


    def forward(self, x):
        '''
        `x` has shape (batch_dim, sequence_length, input_dim) or (N, L, D_in)
        
        The output should have shape (batch_dim, sequence_length, output_dim) or (N, L, D_out).
        '''

        k = self.lin_k(x)
        q = self.lin_q(x)
        v = self.lin_v(x)
        
        output, val = attention(q, k, v)


        return output
    
# test the self-attention module with some random values
torch.manual_seed(48)
input_dim = 512
key_dim = 64
output_dim = 512
self_attn = SelfAttention(input_dim, key_dim, output_dim)
x = torch.randn(4, 10, 512)
output = self_attn(x)
print(f"input shape: {x.shape}")
print(f"output shape: {output.shape}")
print(f'Input: \n{x[0,0,:5]}\nOutput: \n{output[0,0,:5]}\n')

# For self-attention, lets check the permutation-equivariant property, 
# i.e., permute the input sequence and check if the output sequence is also permuted but otherwise the same. 
# This is a nice sanity check that self-attention is working properly.
random_permutation = torch.randperm(x.size(1))
reverse_permutation = torch.zeros_like(random_permutation)
reverse_permutation[random_permutation] = torch.arange(len(random_permutation))
assert torch.all(x[:, random_permutation, :][:, reverse_permutation, :] == x), 'inverse is incorrect'

x_prime = x[:, random_permutation, :] # Permute input
output_prime = self_attn(x_prime)
output_prime_permuted = output_prime[:, reverse_permutation, :]  # Reverse permutation of output
print(f'Does the module exhibit permutation-equivaraince?'
      f' {torch.allclose(output, output_prime_permuted, atol=1e-5, rtol=1)}')
print(f'The following two lines should be the same:')
print(output[-1,-1,:10])
print(output_prime_permuted[-1,-1,:10])

input shape: torch.Size([4, 10, 512])
output shape: torch.Size([4, 10, 512])
Input: 
tensor([-0.1988, -0.3060,  0.6383,  0.5713,  1.2769])
Output: 
tensor([ 8.6836e-02, -4.1725e-02, -2.5798e-01, -9.6712e-02,  3.0022e-05],
       grad_fn=<SliceBackward0>)

Does the module exhibit permutation-equivaraince? True
The following two lines should be the same:
tensor([ 0.1849,  0.3706,  0.1488,  0.0898,  0.2528,  0.4277,  0.4572,  0.1172,
         0.0893, -0.2081], grad_fn=<SliceBackward0>)
tensor([ 0.1849,  0.3706,  0.1488,  0.0898,  0.2528,  0.4277,  0.4572,  0.1172,
         0.0893, -0.2081], grad_fn=<SliceBackward0>)


# Cross Attention module
For cross attention, there will be an first input `x` that will correspond to the query and a second input `y` that will correspond to the keys and values. (In self-attention, `x` and `y` were equal).
This should be the same basic idea except that there is a linear layer to compute `q` from `x` and linear layers to compute `k` and `v` from `y`.

In [53]:
class CrossAttention(nn.Module):
    def __init__(self, x_input_dim, y_input_dim, key_dim, output_dim):
        super().__init__()
        
        self.lin_q = nn.Linear(x_input_dim, key_dim)
        self.lin_k = nn.Linear(y_input_dim, key_dim)
        self.lin_v = nn.Linear(y_input_dim, output_dim)

    def forward(self, x, y):

        k = self.lin_k(y)
        q = self.lin_q(x)
        v = self.lin_v(y)
        output, val = attention(q, k, v)

        
        return output

# test the attention module with some random values
torch.manual_seed(14)
x_input_dim = 512
y_input_dim = 256
key_dim = 64
output_dim = 128
cross_attn = CrossAttention(x_input_dim, y_input_dim, key_dim, output_dim)
x = torch.randn(3, 10, x_input_dim)
y = torch.randn(3, 10, y_input_dim)
output = cross_attn(x, y)
print(f"input shape x and y: {x.shape}, {y.shape}")
print(f"output shape: {output.shape}")
print(f'x\n{x[0,0,:10]}')
print(f'y\n{y[0,0,:10]}')
print(f'output\n{output[0,0,:10]}')

input shape x and y: torch.Size([3, 10, 512]), torch.Size([3, 10, 256])
output shape: torch.Size([3, 10, 128])
x
tensor([-0.0385,  0.9773, -1.4370,  0.8719, -2.1034, -0.2877,  0.3034, -1.9151,
         1.1799,  0.6151])
y
tensor([-2.1565,  0.2397,  0.5872,  0.3950, -0.6114,  0.3489, -0.3467,  0.2792,
        -1.2541,  0.4053])
output
tensor([-0.1544,  0.0847,  0.2329,  0.0549, -0.1424,  0.0711,  0.0105, -0.2139,
        -0.0208, -0.1942], grad_fn=<SliceBackward0>)


## Multi-headed self-attention module
Multi-headed self-attention merely passes the the input to each attention module, concatenates all the outputs, and then applies a linear layer to get the final output.
Implementation of multi-headed attention below.

In [54]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, attn_modules, final_output_dim):
        super().__init__()

        self.attn_mod = nn.ModuleList(attn_modules)

        out_dim = sum([a.lin_v.out_features for a in attn_modules])

        self.lin = nn.Linear(out_dim, final_output_dim)


    def forward(self, x):

        outputs = [a(x) for a in self.attn_mod]

        conc_out = torch.cat(outputs, -1)

        output = self.lin(conc_out)


        return output
    
# test the multi-headed attention module with some random values
torch.manual_seed(10)
input_dim = 256
key_dim = 128 
output_dim = 64 
final_output_dim = 32
num_heads = 8 
attn_modules = [SelfAttention(input_dim, key_dim, output_dim//num_heads) for _ in range(num_heads)]
multi_attn = MultiHeadedAttention(attn_modules, final_output_dim)

x = torch.randn(3, 10, input_dim)
output = multi_attn(x)
print(f"input shape: {x.shape}")
print(f"output shape: {output.shape}")
print(f'x\n{x[0,0,:10]}\noutput\n{output[0,0,:10]}')


input shape: torch.Size([3, 10, 256])
output shape: torch.Size([3, 10, 32])
x
tensor([ 0.7195, -0.3636,  1.3771,  0.3482, -0.0604, -0.3034, -0.0698,  0.2131,
        -0.9736, -0.4651])
output
tensor([ 0.2290, -0.0350,  0.0918, -0.1069, -0.1679, -0.1939, -0.2167,  0.1841,
         0.0546,  0.1668], grad_fn=<SliceBackward0>)
